## Аугментации для улучшения устойчивости модели

| Аугментация                         | Что делает                                | Почему важна                                                             |
|--------------------------------------|-------------------------------------------|--------------------------------------------------------------------------|
| **Random Horizontal Flip**           | Отражает изображение слева-направо         | Люди могут идти в разные стороны                                          |
| **Random Brightness/Contrast**        | Меняет освещенность и контраст             | Условия освещения разные: день, вечер, ночь                               |
| **Random Scale / Crop / Pad**          | Увеличивает/уменьшает человека             | Люди бывают ближе и дальше к камере                                       |
| **Color Jitter**                      | Меняет цветовой оттенок                    | Камеры бывают разные, плюс погодные искажения                             |
| **Random Rotation**                   | Небольшой поворот до ±10°                  | Камера может быть чуть наклонена                                          |
| **Gaussian Blur**                     | Размывает изображение                     | Симуляция плохого качества записи (дождь, снег)                           |
| **CLAHE / Histogram Equalization**    | Вытягивает контраст на сложных снимках     | Улучшение видимости людей при плохой освещенности                         |
| **Random Noise**                      | Добавляет шумы                            | Эмуляция дешевых камер (например, в домофонах)                            |
| **CoarseDropout** | Случайным образом закрывает части изображения прямоугольниками           | Имитирует препятствия на пути обзора (деревья, столбы)                   |


**Как будем строить план аугментации:**
- Берём каждую оригинальную картинку
- На её основе генерируем 7–9 аугментированных вариантов через разные комбинации
- В итоге получаем ~8×559 = около 4500-5000 изображений

In [ ]:
# В Google Colab нет по дефолту ultralytics, надо установить
!pip install ultralytics

In [1]:
import torch
import os
import cv2
import albumentations as A
from tqdm import tqdm
from ultralytics import YOLO
import random
import shutil

In [8]:
# Пути к изображениям
input_dir = './../data/original_images'
output_dir = './../data/augmented_images'

In [3]:
# Сколько новых копий нужно сделать на каждое оригинальное изображение
n_augmentations_per_image = 8  # 8 аугментированных копий + 1 оригинал

In [4]:
# Создаём выходную папку
os.makedirs(output_dir, exist_ok=True)

In [5]:
# Базовый набор аугментаций (рандомные комбинации)
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.2, rotate_limit=10, p=0.5),
    A.GaussianBlur(blur_limit=(3,5), p=0.3),
    A.CLAHE(clip_limit=2.0, p=0.3),
    A.ColorJitter(p=0.4),
    A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=0.3),
    A.CoarseDropout(max_holes=8, max_height=20, max_width=20, min_holes=1, min_height=10, min_width=10, fill_value=0, p=0.3)
], p=1.0)

d:\Documents\ITMO\Semestr_2\dl_in_practice\people-detection\.venv\lib\site-packages\albumentations\core\validation.py:111: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
C:\Users\ghost\AppData\Local\Temp\ipykernel_3840\1773777477.py:10: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_holes, min_height, min_width, fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=8, max_height=20, max_width=20, min_holes=1, min_height=10, min_width=10, fill_value=0, p=0.3)


In [9]:
# Перебираем все изображения
image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

In [11]:
# Выполняем аугментацию
for img_name in tqdm(image_files, desc="Аугментируем изображения"):
    img_path = os.path.join(input_dir, img_name)
    img = cv2.imread(img_path)

    if img is None:
        continue

    # Сохраняем оригинал
    base_filename = os.path.splitext(img_name)[0]
    cv2.imwrite(os.path.join(output_dir, f"{base_filename}_orig.png"), img)

    # Генерируем аугментированные копии
    for i in range(n_augmentations_per_image):
        augmented = transform(image=img)['image']
        aug_filename = f"{base_filename}_aug{i+1}.png"
        cv2.imwrite(os.path.join(output_dir, aug_filename), augmented)

Аугментируем изображения: 100%|██████████| 559/559 [04:53<00:00,  1.90it/s]


**Что мы сделали:**

✅ Разные ракурсы (повороты картинок)

✅ Разное освещение (на случай низкой яркости/контраста)

✅ Размытость (если будут плохие камеры, дождь, снег)

✅ Артефакты на изображении (при наличии объектов типа столбов, веток)

✅ добавление шумов (также на случай плохих камер)

Итого:
- Увеличение датасета ~х9
- Повышение робастности модели
- Подготовка к реальным условиям
- Имитируем сложные сцены

In [2]:
# Пути
input_images_dir = './../data/augmented_images'  # Папка с новыми аугментированными картинками
output_labels_dir = './../data/augmented_labels'  # Куда сохранять разметку

In [3]:
# Создаём папку для сохранения аннотаций
os.makedirs(output_labels_dir, exist_ok=True)

In [4]:
# Загружаем модель YOLO11l
model = YOLO('yolo11x.pt')

In [5]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

True
1
NVIDIA GeForce RTX 4060 Laptop GPU


In [7]:
# Собираем список всех файлов
image_files = [f for f in os.listdir(input_images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
image_paths = [os.path.join(input_images_dir, f) for f in image_files]

batch_size = 8

# Проходим батчами
for i in tqdm(range(0, len(image_paths), batch_size), desc="Прогон инференса и сохранение разметки"):
    batch_paths = image_paths[i:i+batch_size]

    # Прогон инференса
    results = model.predict(
        source=batch_paths,
        imgsz=512,
        batch=batch_size,
        conf=0.3,
        device='auto',
        save=False,
        half=True,
        verbose=False
    )

    # Сохраняем предсказания
    for result in results:
        preds = result.boxes
        img_path = result.path
        img_name = os.path.basename(img_path)

        if preds is not None and preds.xywhn is not None:
            labels = []
            for box, conf, cls in zip(preds.xywhn, preds.conf, preds.cls):
                if conf > 0.3:
                    x_center, y_center, width, height = box.cpu().numpy()
                    class_id = int(cls.item())
                    labels.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

            txt_name = os.path.splitext(img_name)[0] + '.txt'
            with open(os.path.join(output_labels_dir, txt_name), 'w') as f:
                f.write('\n'.join(labels))

Прогон инференса и сохранение разметки:   0%|          | 0/629 [00:00<?, ?it/s]

Прогон инференса и сохранение разметки: 100%|██████████| 629/629 [02:45<00:00,  3.79it/s]


**Что мы сделали:**

✅ Вместо "в лоб" инференса по одной картинке —
мы батчами по 8 изображений быстро прогоняемся через YOLO11x

✅ Вместо огромного imgsz=640 —
мы поставили разумное imgsz=512 (что уменьшает вес операции, почти без потерь в качестве)

✅ Вместо хранения лишних картинок —
мы сохраняем только текстовую разметку .txt (экономия времени и места)

✅ Вместо full precision float32 —
мы включили AMP (half=True), что ускоряет работу нейронки на GPU без видимых потерь точности

Итого:
- Качество разметки останется на высоком уровне
- Скорость разметки В 5–8 раз быстрее
- Потребление VRAM снижено в 2–3 раза

**Теперь разделим все наши данные на:**

- train (~80%)

- val (~10%)

- test (~10%)

In [8]:
# Пути к исходным данным
base_images_dir = './../data/augmented_images'
base_labels_dir = './../data/augmented_labels'

# Куда будем сохранять
output_base = './../data/final_dataset'

# Создание папок для каждого сплита
splits = ['train', 'val', 'test']
for split in splits:
    os.makedirs(os.path.join(output_base, 'images', split), exist_ok=True)
    os.makedirs(os.path.join(output_base, 'labels', split), exist_ok=True)

In [9]:
# Получаем список всех изображений
image_files = [f for f in os.listdir(base_images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Перемешиваем
random.seed(42)
random.shuffle(image_files)

# Делим на train / val / test
n_total = len(image_files)
n_train = int(0.8 * n_total)
n_val = int(0.1 * n_total)
n_test = n_total - n_train - n_val

train_files = image_files[:n_train]
val_files = image_files[n_train:n_train+n_val]
test_files = image_files[n_train+n_val:]

dataset_split = {
    'train': train_files,
    'val': val_files,
    'test': test_files
}

# Копирование файлов
for split, files in dataset_split.items():
    for img_name in tqdm(files, desc=f"Копируем {split}"):
        img_src = os.path.join(base_images_dir, img_name)
        label_src = os.path.join(base_labels_dir, os.path.splitext(img_name)[0] + '.txt')

        img_dst = os.path.join(output_base, 'images', split, img_name)
        label_dst = os.path.join(output_base, 'labels', split, os.path.splitext(img_name)[0] + '.txt')

        # Копируем картинку
        shutil.copyfile(img_src, img_dst)

        # Копируем разметку
        if os.path.exists(label_src):
            shutil.copyfile(label_src, label_dst)
        else:
            print(f"Нет разметки для: {img_name}")

Копируем test: 100%|██████████| 504/504 [00:01<00:00, 325.67it/s]
